# Warsztaty Python w Data Science

---
## Inżynieria Cech (Feature Engineering) - część 3 z 3  

- ### Czy Data Scientiści mają problem ?
- ### Zautomatyzowana eksploracja danych
- ### Zautomatyzowana inżynieria cech
- ### Redukcja wymiarów

---
## Czy Data Scientiści mają problem ?

https://towardsdatascience.com/data-scientists-will-be-extinct-in-10-years-a6e5dd77162b
> ## __Data Scientists Will be Extinct in 10 Years__
> ### __And Why It’s Not a Bad Thing__



> #### Going forward, the skill set collectively known as data science will be borne by a new generation of data savvy business specialists 
> #### and subject matter experts who are able to imbue analysis with their deep domain knowledge, irrespective of whether they can code or not.

---

## Zautomatyzowana eksploracja danych

In [2]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

df = pd.DataFrame(np.random.rand(100, 5), columns=["a", "b", "c", "d", "e"])
df

,a,b,c,d,e
0,0.662079,0.852037,0.736949,0.393183,0.016716
1,0.912391,0.930994,0.064348,0.160981,0.083272
2,0.448697,0.477549,0.905583,0.400441,0.866001
3,0.540855,0.197583,0.415546,0.911331,0.126282
4,0.404920,0.636981,0.692017,0.006466,0.983326
...,...,...,...,...,...
95,0.741042,0.604900,0.483492,0.342664,0.852608
96,0.804350,0.081746,0.736781,0.062543,0.746215
97,0.040699,0.772143,0.106968,0.142732,0.479943
98,0.227047,0.521551,0.080177,0.001695,0.954593


In [3]:
profile = ProfileReport(df, title="Pandas Profiling Report", dark_mode=True)

In [4]:
profile.to_notebook_iframe()

In [5]:
profile.to_widgets()

---
## Zautomatyzowana inżynieria cech

### Deep Feature Synthesis

> ### In 2015, researchers at MIT presented the Deep Feature Synthesis algorithm and demonstrated its effectiveness in online data science competitions where it beat `615` of `906` human teams.

https://groups.csail.mit.edu/EVO-DesignOpt/groupWebSite/uploads/Site/DSAA_DSM_2015.pdf

In [6]:
import featuretools as ft
es = ft.demo.load_mock_customer(return_entityset=True)
es

Entityset: transactions
  DataFrames:
    transactions [Rows: 500, Columns: 6]
    products [Rows: 5, Columns: 3]
    sessions [Rows: 35, Columns: 5]
    customers [Rows: 5, Columns: 5]
  Relationships:
    transactions.product_id -> products.product_id
    transactions.session_id -> sessions.session_id
    sessions.customer_id -> customers.customer_id

In [7]:
es.dataframe_dict.keys()

dict_keys(['transactions', 'products', 'sessions', 'customers'])

In [8]:
es.dataframe_dict['customers']

,customer_id,zip_code,join_date,birthday,_ft_last_time
5,5,60091,2010-07-17 05:27:50,1984-07-28,2014-01-01 08:09:40
4,4,60091,2011-04-08 20:08:14,2006-08-15,2014-01-01 05:31:30
1,1,60091,2011-04-17 10:48:33,1994-07-18,2014-01-01 07:26:20
3,3,13244,2011-08-13 15:42:34,2003-11-21,2014-01-01 09:00:35
2,2,13244,2012-04-15 23:31:04,1986-08-18,2014-01-01 08:23:45


In [9]:
df = es.dataframe_dict['transactions']
df

,transaction_id,session_id,transaction_time,product_id,amount,_ft_last_time
298,298,1,2014-01-01 00:00:00,5,127.64,2014-01-01 00:00:00
2,2,1,2014-01-01 00:01:05,2,109.48,2014-01-01 00:01:05
308,308,1,2014-01-01 00:02:10,3,95.06,2014-01-01 00:02:10
116,116,1,2014-01-01 00:03:15,4,78.92,2014-01-01 00:03:15
371,371,1,2014-01-01 00:04:20,3,31.54,2014-01-01 00:04:20
...,...,...,...,...,...,...
112,112,35,2014-01-01 08:56:15,5,55.42,2014-01-01 08:56:15
111,111,35,2014-01-01 08:57:20,3,34.87,2014-01-01 08:57:20
276,276,35,2014-01-01 08:58:25,1,10.94,2014-01-01 08:58:25
266,266,35,2014-01-01 08:59:30,5,19.86,2014-01-01 08:59:30


In [10]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report",  dark_mode=True)
profile

In [11]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="customers",
                                      agg_primitives=["count"],
                                      trans_primitives=["month"],
                                      max_depth=1)
feature_matrix

,zip_code,COUNT(sessions),MONTH(birthday),MONTH(join_date)
customer_id,,,,
5,60091,6,7,7
4,60091,8,8,4
1,60091,8,7,4
3,13244,6,11,8
2,13244,7,8,4


In [12]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="customers",
                                      agg_primitives=["mean", "sum", "mode"],
                                      trans_primitives=["month", "hour"],
                                      max_depth=2)
feature_matrix

,zip_code,MODE(sessions.device),MEAN(transactions.amount),MODE(transactions.product_id),SUM(transactions.amount),HOUR(birthday),HOUR(join_date),MONTH(birthday),MONTH(join_date),MEAN(sessions.MEAN(transactions.amount)),MEAN(sessions.SUM(transactions.amount)),MODE(sessions.HOUR(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.MONTH(session_start)),SUM(sessions.MEAN(transactions.amount)),MODE(transactions.sessions.device)
customer_id,,,,,,,,,,,,,,,,
5,60091,mobile,80.375443,5,6349.66,0,5,7,7,78.705187,1058.276667,0,3,1,472.231119,mobile
4,60091,mobile,80.070459,2,8727.68,0,20,8,4,81.207189,1090.960000,1,1,1,649.657515,mobile
1,60091,mobile,71.631905,4,9025.62,0,10,7,4,72.774140,1128.202500,6,4,1,582.193117,mobile
3,13244,desktop,67.060430,1,6236.62,0,15,11,8,67.539577,1039.436667,5,1,1,405.237462,desktop
2,13244,desktop,77.422366,4,7200.28,0,23,8,4,78.415122,1028.611429,3,3,1,548.905851,desktop


In [13]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_dataframe_name="customers",
                                      agg_primitives=["mean", "sum", "mode"],
                                      trans_primitives=["month", "hour"],
                                      max_depth=3)
feature_matrix

,zip_code,MODE(sessions.device),MEAN(transactions.amount),MODE(transactions.product_id),SUM(transactions.amount),HOUR(birthday),HOUR(join_date),MONTH(birthday),MONTH(join_date),MEAN(sessions.MEAN(transactions.amount)),MEAN(sessions.SUM(transactions.amount)),MODE(sessions.HOUR(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.MONTH(session_start)),SUM(sessions.MEAN(transactions.amount)),MODE(transactions.HOUR(transaction_time)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.sessions.device),MODE(sessions.MODE(transactions.HOUR(transaction_time))),MODE(sessions.MODE(transactions.MONTH(transaction_time)))
customer_id,,,,,,,,,,,,,,,,,,,,
5,60091,mobile,80.375443,5,6349.66,0,5,7,7,78.705187,1058.276667,0,3,1,472.231119,7,1,mobile,0,1
4,60091,mobile,80.070459,2,8727.68,0,20,8,4,81.207189,1090.960000,1,1,1,649.657515,5,1,mobile,2,1
1,60091,mobile,71.631905,4,9025.62,0,10,7,4,72.774140,1128.202500,6,4,1,582.193117,6,1,mobile,6,1
3,13244,desktop,67.060430,1,6236.62,0,15,11,8,67.539577,1039.436667,5,1,1,405.237462,8,1,desktop,8,1
2,13244,desktop,77.422366,4,7200.28,0,23,8,4,78.415122,1028.611429,3,3,1,548.905851,3,1,desktop,3,1


In [14]:
feature_matrix[['MODE(sessions.HOUR(session_start))']]

,MODE(sessions.HOUR(session_start))
customer_id,
5,0
4,1
1,6
3,5
2,3


In [15]:
ft.describe_feature(feature_defs[9])

'The average of the average of the "amount" of all instances of "transactions" for each "session_id" in "sessions" of all instances of "sessions" for each "customer_id" in "customers".'

In [16]:
import pandas as pd
import featuretools as ft

from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

from featuretools.primitives import NaturalLanguage
from featuretools.demo.flight import load_flight

es = load_flight(nrows=50)
es

Entityset: Flight Data
  DataFrames:
    trip_logs [Rows: 50, Columns: 21]
    flights [Rows: 6, Columns: 9]
    airlines [Rows: 1, Columns: 1]
    airports [Rows: 4, Columns: 3]
  Relationships:
    trip_logs.flight_id -> flights.flight_id
    flights.carrier -> airlines.carrier
    flights.dest -> airports.dest

In [17]:
fm, features = ft.dfs(entityset=es,
                          target_dataframe_name="trip_logs",
                          cutoff_time=pd.DataFrame({
                              'trip_log_id':[30, 1, 2, 3, 4],
                              'time':pd.to_datetime(['2016-09-22 00:00:00']*5)
                                                  }),
                          trans_primitives=[],
                          agg_primitives=[],
                          max_depth=2)
fm

,flight_id,dep_delay,taxi_out,taxi_in,arr_delay,diverted,air_time,distance,carrier_delay,weather_delay,national_airspace_delay,security_delay,late_aircraft_delay,canceled,flights.origin,flights.origin_city,flights.origin_state,flights.dest,flights.distance_group,flights.carrier,flights.flight_num,flights.airports.dest_city,flights.airports.dest_state
trip_log_id,,,,,,,,,,,,,,,,,,,,,,,
30,AA-494:RSW->CLT,NaN,NaN,NaN,NaN,<NA>,NaN,600.0,NaN,NaN,NaN,NaN,NaN,<NA>,RSW,"Fort Myers, FL",FL,CLT,3,AA,494,"Charlotte, NC",NC
1,AA-494:CLT->PHX,NaN,NaN,NaN,NaN,<NA>,NaN,1773.0,NaN,NaN,NaN,NaN,NaN,<NA>,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
2,AA-494:CLT->PHX,NaN,NaN,NaN,NaN,<NA>,NaN,1773.0,NaN,NaN,NaN,NaN,NaN,<NA>,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
3,AA-494:CLT->PHX,NaN,NaN,NaN,NaN,<NA>,NaN,1773.0,NaN,NaN,NaN,NaN,NaN,<NA>,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
4,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
remove_highly_null_features(fm)

,flight_id,distance,flights.origin,flights.origin_city,flights.origin_state,flights.dest,flights.distance_group,flights.carrier,flights.flight_num,flights.airports.dest_city,flights.airports.dest_state
trip_log_id,,,,,,,,,,,
30,AA-494:RSW->CLT,600.0,RSW,"Fort Myers, FL",FL,CLT,3,AA,494,"Charlotte, NC",NC
1,AA-494:CLT->PHX,1773.0,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
2,AA-494:CLT->PHX,1773.0,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
3,AA-494:CLT->PHX,1773.0,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
remove_highly_null_features(fm, pct_null_threshold=.2)

""
trip_log_id
30
1
2
3
4


In [20]:
new_fm, new_features = remove_single_value_features(fm, features=features, count_nan_as_value=True)
new_fm

,flight_id,distance,flights.origin,flights.origin_city,flights.origin_state,flights.dest,flights.distance_group,flights.carrier,flights.flight_num,flights.airports.dest_city,flights.airports.dest_state
trip_log_id,,,,,,,,,,,
30,AA-494:RSW->CLT,600.0,RSW,"Fort Myers, FL",FL,CLT,3,AA,494,"Charlotte, NC",NC
1,AA-494:CLT->PHX,1773.0,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
2,AA-494:CLT->PHX,1773.0,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
3,AA-494:CLT->PHX,1773.0,CLT,"Charlotte, NC",NC,PHX,8,AA,494,"Phoenix, AZ",AZ
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
